# Starting with Psi4 and Gamess from within a Notebook

To begin with I will explore using Psi4 and Gamess and calculation engines from within a Jupyter notebook. 

I am using a conda envirinmnet with the following installed (in addition to the usual requirements)

conda install rdkit psi4 jupyter py3Dmol
pip install pygamess chemtools

I will be seriously stealing ideas and code from the following sources as I proceed.

https://github.com/Mishima-syk/psikit
- wrapper for Psi4 in notebooks

https://github.com/kzfm/pygamess
- scripts to directly call Gamess

https://github.com/tommason14/autochem
- classes to create input files and parse output files in gamess

https://github.com/STW59/simulation-scripts
- scripts to run gamess

https://github.com/mrobinson-hub/FMO-Scripts/tree/master
- scripts for using Gamess and PSI4. Seems specialized.

https://github.com/Cipulot/GAMESS-Interface
- looks interesting. A GUI to interpret gamess results for Avogadro?

https://github.com/nar-n/Run-Gamess-in-Python
- will it do what it says? A simple short script. First place to learn?

https://github.com/nidemidovich/gamess-calculations
- python wrapper for gamess. Uses ASE

## Initial work

all from 

https://github.com/Jammyzx1/psi4-nb/blob/main/psi4_intoductory_methods.ipynb












In [1]:
### use psi4 conda environment
### https://github.com/Jammyzx1/psi4-nb/blob/main/psi4_intoductory_methods.ipynb

from typing import Union, Tuple
import psi4
import os
import numpy as np
import logging

logging.basicConfig(format="%(message)s")
log = logging.getLogger(__name__)
log.setLevel("INFO")

print('done')

done


In [44]:
dir(logging)

['BASIC_FORMAT',
 'BufferingFormatter',
 'CRITICAL',
 'DEBUG',
 'ERROR',
 'FATAL',
 'FileHandler',
 'Filter',
 'Filterer',
 'Formatter',
 'GenericAlias',
 'Handler',
 'INFO',
 'LogRecord',
 'Logger',
 'LoggerAdapter',
 'Manager',
 'NOTSET',
 'NullHandler',
 'PercentStyle',
 'PlaceHolder',
 'RootLogger',
 'StrFormatStyle',
 'StreamHandler',
 'StringTemplateStyle',
 'Template',
 'WARN',
 'WARNING',
 '_STYLES',
 '_StderrHandler',
 '__all__',
 '__author__',
 '__builtins__',
 '__cached__',
 '__date__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__status__',
 '__version__',
 '_acquireLock',
 '_addHandlerRef',
 '_after_at_fork_child_reinit_locks',
 '_at_fork_reinit_lock_weakset',
 '_checkLevel',
 '_defaultFormatter',
 '_defaultLastResort',
 '_handlerList',
 '_handlers',
 '_is_internal_frame',
 '_levelToName',
 '_lock',
 '_logRecordFactory',
 '_loggerClass',
 '_nameToLevel',
 '_register_at_fork_reinit_lock',
 '_releaseLock',
 '_removeHandle

In [41]:
psi4.core.print_global_options()






  Global Options:
  ----------------------------------------------------------------------------
  ABCD                               => NEW               
  ACTIVE                             => [  ]              
  ACTIVE_NAT_ORBS                    => [  ]              
  ADD_AUXILIARY_BONDS                => TRUE              
  AIO_CPHF                           => FALSE             
  AIO_DF_INTS                        => FALSE             
  ALGORITHM                          => SIMULTANEOUS      
  ANALYZE                            => FALSE             
  AO_BASIS                           => DISK              
  AUXILIARY_BOND_FACTOR              => 2.5               
  AVG_STATES                         => [  ]              
  AVG_WEIGHTS                        => [  ]              
  A_RAS3_MAX                         => -1                
  BASIS                              => (empty)           
  BASIS_GUESS                        => FALSE             
  BASIS_RELATIVI

In [39]:
print(a)


None


In [2]:
psi4.set_options(
    {
        "SAVE_OPTIMIZATION": True,
        "MAXITER": 100,
        "GEOM_MAXITER": 100,
        "FULL_HESS_EVERY": 5,
        "PRINT": 2,
        "GUESS": "sad",
        "REFERENCE": "uhf",
        "SCF_TYPE": "direct",
    }
)

In [5]:
psi4.core.print_options()



  Module  Options:
  ----------------------------------------------------------------------------



In [40]:
psi4.core.clean_options()        ### Reset options set with psi4.set_options

In [6]:
os.getcwd()

'/Users/blink/github/TryingPsi4andMore'

In [3]:
psi4.set_memory("1GB")
psi4.set_output_file("p4_output.out", append=False, loglevel=20, print_header=True, inherit_loglevel=True)
psi4.core.set_num_threads(4)


  Memory set to 953.674 MiB by Python driver.


In [4]:
molecule = psi4.geometry(
    """
    0 1
    O     0.00000     0.64670    -0.01863
    H     0.76026     0.61622    -0.62453
    H    -0.76026     0.61622    -0.62453
    units angstrom
    symmetry c1
    """ 
)

In [5]:
energy = psi4.energy("hf/cc-pvdz")
log.info("The energy for this configuration is {:.7f} Hartree".format(energy))

The energy for this configuration is -76.0256113 Hartree


In [10]:
energy

-76.02561133372092

In [6]:
molecule = psi4.geometry(
    """
    0 1
    O     0.00000     0.64670    -0.01863
    H     0.76026     0.61622    -0.62453
    H    -0.76026     0.61622    -0.62453
    units angstrom
    symmetry c1
    """ 
)

In [7]:
opt_energy = psi4.optimize("hf/cc-pvdz")
log.info("The optimized energy for this configuration is {:.7f} Hartree".format(opt_energy))

The optimized energy for this configuration is -76.0270534 Hartree


Optimizer: Optimization complete!


In [8]:
log.info("Difference in energy from initial conformation to optimized conformation {:.7f} Hartree".format(energy - opt_energy))

Difference in energy from initial conformation to optimized conformation 0.0014421 Hartree


In [32]:
psi4.core.clean()

In [9]:
def psi4_optimize(initial_iterations: int = 2,
                  increment_iteration: int = 2,
                  max_loop_count: int = 20,
                  molecule: Union[psi4.core.Molecule, None] = None,
                  method: str = "hf/cc-pvdz"
                 ) -> Tuple[float, psi4.core.Wavefunction, dict]:
    """
    General function to run Psi4 optimization and restart a max of max_loop_count times incrementing the number
    of iterations by increment_iteration from initial_iterations each loop
    :param initial_iterations: integer the number of initial iterations
    :param increment_iteration: integer the number of iteration increment by each time it fails to converge
    :param max_loop_count: integer the maximum number of times to increment for convergence incomplete
    :param molecule: psi4.core.Molecule the molecule object or None if given will save last geometry of each unconverged loop
    :method: string define the quantum chem optimization method eg hf/cc-pvdz
    :return: Tuple[float, psi4.core.Wavefunction, list] the energy, the wavefunction object and the history of conformations
    """
    
    log = logging.getLogger(__name__)
    
    unconv = True

    psi4.set_options(
        {
            "MAXITER": initial_iterations,
            "GEOM_MAXITER": initial_iterations
        }
    )
    
    loop_count = 0
    
    if molecule is not None:
        molecule_traj = []

    while unconv is True:
        
        try:
            
            log.info("\ncount {}\n".format(loop_count))
            opt_energy, wfn, traj = psi4.optimize(method, return_wfn=True, return_history=True)
            log.info("The optimized energy for this configuration is {:.6f} Hartree".format(opt_energy))
            unconv = False
            
        except psi4.driver.ConvergenceError as cerr:
            
            log.warning("Geometry unconverged will try restarting")
            if molecule is not None:
                bohr_coor = molecule.geometry()
                bohr_coor.scale(psi4.constants.bohr2angstroms)
                molecule_traj.append(bohr_coor.to_array())
            unconv_wfn = cerr.wfn
            unconv_wfn.to_file(unconv_wfn.get_scratch_filename(180))
            psi4.set_options(
                {
                    "GUESS": "read",
                    "MAXITER": initial_iterations + increment_iteration,
                    "GEOM_MAXITER": initial_iterations + increment_iteration
                }
            )
            
            loop_count = loop_count + 1

            if max_loop_count <= loop_count:
                log.error("Unconverged in maximum number of loops")
                raise cerr
                
    if molecule is not None:
        traj = {"coordinates": tuple(np.array(molecule_traj))}

    return opt_energy, wfn, traj

In [10]:
molecule = psi4.geometry(
    """
    0 1
    O     0.00000     0.74670    -0.01863
    H     0.76026     0.61622    -0.62453
    H    -0.76026     0.61622    -0.62453
    units angstrom
    symmetry c1
    """ 
)

psi4.set_memory("1GB")
psi4.set_output_file("p4_output.out", append=False, loglevel=20, print_header=True, inherit_loglevel=True)
psi4.core.set_num_threads(4)

#Include optional molecule=molecule to get a trajectory of the final geometry of each loop inplace of the psi4 history
energy, wfn, traj = psi4_optimize(initial_iterations=10,
                                  increment_iteration=10,
                                  max_loop_count=20,
                                  method="hf/cc-pvdz",
                                 )

log.info("Optimized energy {:.7f}".format(energy))



count 0

Geometry unconverged will try restarting

count 1

The optimized energy for this configuration is -76.027054 Hartree
Optimized energy -76.0270535


Optimizer: Optimization complete!


In [12]:
oeprops = psi4.core.OEProp(wfn)
oeprops.add("DIPOLE")
oeprops.add("QUADRUPOLE")
oeprops.add("MULLIKEN_CHARGES")
oeprops.add("MULTIPOLE(4)")
oeprops.add("ESP_AT_NUCLEI")
oeprops.add("MO_EXTENTS")
oeprops.add("LOWDIN_CHARGES")
oeprops.add("WIBERG_LOWDIN_INDICES")
oeprops.add("MAYER_INDICES")
oeprops.add("NO_OCCUPATIONS")
oeprops.compute()

In [13]:
properties = wfn.variables()
properties["SCF TOTAL ENERGIES"]

array([-76.02705349, -76.02705350, -76.02705350, -76.02705350,
       -76.02705350, -76.02705350])

In [13]:
for a in properties.keys(): print(a)

CURRENT ENERGY
CURRENT REFERENCE ENERGY
DD SOLVATION ENERGY
ESP AT CENTER 1
ESP AT CENTER 2
ESP AT CENTER 3
HF KINETIC ENERGY
HF POTENTIAL ENERGY
HF TOTAL ENERGY
HF VIRIAL RATIO
NUCLEAR REPULSION ENERGY
ONE-ELECTRON ENERGY
PCM POLARIZATION ENERGY
PE ENERGY
SCF ITERATION ENERGY
SCF ITERATIONS
SCF TOTAL ENERGY
TWO-ELECTRON ENERGY
CURRENT DIPOLE
CURRENT GRADIENT
DIPOLE
HEXADECAPOLE
HF TOTAL GRADIENT
LOWDIN CHARGES
MAYER INDICES
MULLIKEN CHARGES
OCTUPOLE
QUADRUPOLE
SCF DIPOLE
SCF TOTAL ENERGIES
SCF TOTAL GRADIENT
WIBERG LOWDIN INDICES


In [14]:
a = !grep "B(2,1,3)         = " p4_output.out
a[0].split(" ")[-1]

'102.822228'

In [10]:
from grepfunc import grep

output_file = "p4_output.log"
infile = open(output_file, 'r')                 # open file objects a read only

search_string = "B\(2,1,3\)         ="          # must escape special characters with "\"
grep(infile, search_string, i=False, n=False)   #i: ignore case, n:include line numbers


ModuleNotFoundError: No module named 'grepfunc'

In [2]:
import cclib

output_file = "p4_output.out"
parser = cclib.io.ccopen(output_file)
data = parser.parse()
print("There are %i atoms and %i MOs" % (data.natom, data.nmo))

There are 3 atoms and 24 MOs


In [3]:
dir(data)

['OPT_DONE',
 'OPT_NEW',
 'OPT_UNCONVERGED',
 'OPT_UNKNOWN',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_attrlist',
 '_dictsofarrays',
 '_dictsofdicts',
 '_intarrays',
 '_listsofarrays',
 'arrayify',
 'atombasis',
 'atomcoords',
 'atommasses',
 'atomnos',
 'charge',
 'check_values',
 'closed_shell',
 'converged_geometries',
 'coreelectrons',
 'geotargets',
 'geovalues',
 'getattributes',
 'grads',
 'hessian',
 'homos',
 'listify',
 'metadata',
 'moenergies',
 'moments',
 'mosyms',
 'mult',
 'natom',
 'nbasis',
 'nelectrons',
 'new_geometries',
 'nmo',
 'optdone',
 'optstatus',
 'scfenergies',
 'scftargets',
 'scfvalues',
 'setattr

In [27]:
data.scfvalues

[array([[-7.54920e+01,  0.00000e+00],
        [-4.59620e-01,  1.74113e-02],
        [-5.36142e-02,  1.01434e-02],
        [-1.98256e-02,  1.08596e-03],
        [-4.76892e-04,  2.20709e-04],
        [-2.16622e-05,  3.99282e-05],
        [-9.82508e-07,  6.06172e-06],
        [-2.43505e-08,  7.98511e-07],
        [-7.60256e+01,  7.15409e-06],
        [-1.31452e-08,  1.16164e-06],
        [-4.17913e-10,  2.50419e-07]])]